In [4]:
import cv2
import numpy as np
import math

IMAGE_PATH = "/content/test_image_marker3.png"

##Данные Наиля
# K = np.array([
#     [2554.414833, 0.0, 681.885900],
#     [0.0, 2557.461333, 597.316448],
#     [0.0, 0.0, 1.0]
# ], dtype=np.float32)

# dist = np.array([-0.046462, 0.284386, 0.003340, -0.001596, 0.000000], dtype=np.float32)

#мои Данные
K = np.array([
    [2585.290415, 0.0, 716.336125],
    [0.0, 2582.119597, 620.850967],
    [0.0, 0.0, 1.0]
], dtype=np.float32)

dist = np.array([-0.063212, 0.479756, 0.007614, 0.003480, 0.000000], dtype=np.float32)

# #Данные Тимура
# K = np.array([
#     [2554.307419, 0.0, 702.979723],
#     [0.0, 2555.495155, 641.800488],
#     [0.0, 0.0, 1.0]
# ], dtype=np.float32)

# dist = np.array([-0.111646, 0.726067, 0.012216, 0.002722, 0.000000], dtype=np.float32)

SIZE_M1 = 100.0
SIZE_M2 = 28.0

P_M1_base = np.array([-162.0, -274.0, 2.5])

image = cv2.imread(IMAGE_PATH)
if image is None:
    raise FileNotFoundError(IMAGE_PATH)

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
detector = cv2.aruco.ArucoDetector(aruco_dict, cv2.aruco.DetectorParameters())
corners, ids, _ = detector.detectMarkers(image)

if ids is None or 3 not in ids or 15 not in ids:
    raise ValueError("Нет маркеров ID=3 и ID=15")

marker_corners = {}
for i, id_val in enumerate(ids.flatten()):
    marker_corners[id_val] = corners[i][0]

def estimate_marker_pose(corners_2d, size):
    half = size / 2.0
    obj = np.array([
        [-half,  half, 0],
        [ half,  half, 0],
        [ half, -half, 0],
        [-half, -half, 0]
    ], dtype=np.float32)
    success, rvec, tvec = cv2.solvePnP(obj, corners_2d, K, dist, flags=cv2.SOLVEPNP_IPPE_SQUARE)
    if not success:
        success, rvec, tvec = cv2.solvePnP(obj, corners_2d, K, dist, flags=cv2.SOLVEPNP_ITERATIVE)
    if not success:
        raise RuntimeError("solvePnP failed")
    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.flatten()
    return T

T_cam_M1 = estimate_marker_pose(marker_corners[3], SIZE_M1)
T_cam_M2 = estimate_marker_pose(marker_corners[15], SIZE_M2)

T_base_M1 = np.eye(4)
T_base_M1[:3, 3] = P_M1_base

T_base_cam = T_base_M1 @ np.linalg.inv(T_cam_M1)

T_base_M2 = T_base_cam @ T_cam_M2

T_base_M2[2, 3] = P_M1_base[2] + 50.0

R = T_base_M2[:3, :3]
yaw_rad = math.atan2(R[1, 0], R[0, 0])
yaw_deg = -math.degrees(yaw_rad)

print("Результат:")
print(f"X: {T_base_M2[0, 3]:.2f} мм")
print(f"Y: {T_base_M2[1, 3]:.2f} мм")
print(f"Z: {T_base_M2[2, 3]:.2f} мм")
print(f"YAW: {yaw_deg:.2f} градусов")

Результат:
X: -94.67 мм
Y: -455.36 мм
Z: 52.50 мм
YAW: 70.11 градусов
